<a href="https://colab.research.google.com/github/abinhack/ASL_to_Text/blob/master/ASL_to_text_using_ConvLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Run this piece of code to download the dataset**


In [3]:
!wget https://download.microsoft.com/download/3/c/a/3ca92c78-1c4a-4a91-a7ee-6980c1d242ec/MS-ASL.zip
!unzip 'MS-ASL.zip'

--2020-03-23 12:39:01--  https://download.microsoft.com/download/3/c/a/3ca92c78-1c4a-4a91-a7ee-6980c1d242ec/MS-ASL.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.46.196.215, 2600:1402:800:2ac::e59, 2600:1402:800:2a8::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.46.196.215|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2022053 (1.9M) [application/octet-stream]
Saving to: ‘MS-ASL.zip’

MS-ASL.zip          100%[===================>]   1.93M  --.-KB/s    in 0.1s    

2020-03-23 12:39:02 (17.2 MB/s) - ‘MS-ASL.zip’ saved [2022053/2022053]

Archive:  MS-ASL.zip
  inflating: MS-ASL/C-UDA-0.1_annotated_discussion.pdf  
  inflating: MS-ASL/MSASL_classes.json  
  inflating: MS-ASL/MSASL_synonym.json  
  inflating: MS-ASL/MSASL_test.json  
  inflating: MS-ASL/MSASL_train.json  
  inflating: MS-ASL/MSASL_val.json   
  inflating: MS-ASL/README.md        


In [2]:

!pip3 uninstall scipy
!pip3 install scipy==1.1.0
!pip install pafy
!pip install youtube-dl

Uninstalling scipy-1.4.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/scipy-1.4.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/scipy/*
Proceed (y/n)? y
  Successfully uninstalled scipy-1.4.1
     |████████████████████████████████| 31.2MB 1.4MB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 1.8MB 2.8MB/s 


In [0]:
import json
import pandas as pd
import pickle
import os
import cv2
import numpy as np
import scipy
from scipy.misc import imread
import  pickle
import random
import os
import matplotlib.pyplot as plt
import pafy
from google.colab.patches import cv2_imshow
import torch
IMAGE_SIZE =400
NO_OF_WORDS = 5

In [5]:
print("Files in the  current Directory",os.listdir())
print("Files in the MS-ASL directory",os.listdir('MS-ASL'))

Files in the  current Directory ['.config', 'MS-ASL.zip', 'MS-ASL', 'sample_data']
Files in the MS-ASL directory ['C-UDA-0.1_annotated_discussion.pdf', 'MSASL_train.json', 'README.md', 'MSASL_val.json', 'MSASL_classes.json', 'MSASL_test.json', 'MSASL_synonym.json']


**Pre Process Training Data**


In [6]:

with open('MS-ASL/MSASL_train.json') as f:
  train_data = json.load(f)
train_df = pd.DataFrame(train_data)
train_df.head()

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review
0,match [light-a-MATCH],match,0.0,0,0,0,83,match light-a-MATCH,830,360.0,30.00,2.767,https://www.youtube.com/watch?v=C37R_Ix8-qs,match,"[0.05754461884498596, 0.21637457609176636, 1.0...",640.0,NaN
1,FAIL,fail,0.0,0,-1,0,74,FAIL,542,360.0,25.00,2.960,https://www.youtube.com/watch?v=PIsUJl8BN_I,fail,"[0.0657794177532196, 0.16717177629470825, 0.93...",480.0,NaN
2,laugh,laugh,0.0,4,26,0,31,SignSchool Laugh with Legs 2,312,360.0,29.97,1.034,www.youtube.com/watch?v=9FdHlMOnVjg,laugh,"[0.13188594579696655, 0.32334136962890625, 1.0...",640.0,NaN
3,BOOK,book,0.0,0,-1,0,66,BOOK(3),38,360.0,25.00,2.640,https://www.youtube.com/watch?v=J7tP98oDxqE,book,"[0.05569887161254883, 0.25173279643058777, 0.9...",480.0,NaN
4,sign-language,sign language,0.0,0,-1,0,75,SIGN-LANGUAGE-S-CLAW-F,848,360.0,29.97,2.502,www.youtube.com/watch?v=N2mG9ZKjrGA,sign language,"[0.03904399275779724, 0.24198183417320251, 1.0...",640.0,NaN


In [0]:
sample_labels  = train_df['text'].value_counts()[0:NO_OF_WORDS].index.tolist() #training it on top 5 sample labels
train_df = train_df[train_df['text'].isin(sample_labels)]
train_df.reset_index(inplace=True)


In [0]:

avg_frame_train = process_entire_df(train_df)

Total No of rows to be processed 264
processing row-  0


In [0]:
sample_labels  = train_df['text'].value_counts()[0:NO_OF_WORDS].index.tolist() #training it on top 5 sample labels
train_df = train_df[train_df['text'].isin(sample_labels)]
train_df.reset_index(inplace=True)
avg_frame_train = process_entire_df(train_df)

data_classes = train_df['text'].unique().tolist()
train_df['text_no'] = train_df['text'].apply(data_classes.index)
train_labels = train_df['text_no'].tolist()
train_data_processed = (avg_frame_train,train_labels)
pickle.dump(train_data_processed,open("train_data_processed.pkl",'wb'))

NameError: ignored

**Validation Dataset Preprocessing**

In [0]:
with open('MS-ASL/MSASL_val.json') as f:
  val_data = json.load(f)
val_df = pd.DataFrame(val_data)
val_df.head()

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review
0,absent,absent,0.0,114,-1,0,37,ASL ABSENT,837,360.0,28.971,1.277,https://www.youtube.com/watch?v=ri3NrdgfAtE,absent,"[0.2189638912677765, 0.00856819748878479, 0.99...",202.0,NaN
1,help,help,0.0,76,42,0,110,help 2,50,360.0,29.970,3.670,www.youtube.com/watch?v=l31UXgChCS4,help,"[0.05037271976470947, 0.2994125783443451, 1.0,...",640.0,NaN
2,come on,come on,0.0,114,-1,0,41,asl come on,889,360.0,25.000,1.640,https://www.youtube.com/watch?v=pt9bV_EvcaU,come on,"[0.08946925401687622, 0.17948511242866516, 0.9...",480.0,NaN
3,LANGUAGE,language,0.0,3,-1,0,56,LANGUAGE(3),513,360.0,15.000,3.733,https://www.youtube.com/watch?v=-j1wozf6o9w,language,"[0.1770857870578766, 0.003668457269668579, 1.0...",480.0,NaN
4,confused,confused,0.0,53,-1,0,95,ASL Confused,272,360.0,29.969,3.170,https://www.youtube.com/watch?v=y8tHmOQcCwU,confused,"[0.06262531876564026, 0.2099878489971161, 1.0,...",640.0,NaN


In [0]:
val_df = val_df[val_df['text'].isin(sample_labels)]
val_df.reset_index(inplace=True)
avg_frame_val = process_entire_df(val_df)
val_df['text_no'] = val_df['text'].apply(data_classes.index)
val_labels = val_df['text_no'].tolist()
val_data_processed = (avg_frame_val,val_labels)
pickle.dump(val_data_processed,open("val_data_processed.pkl",'wb'))

Total No of rows to be processed 66
processing row-  89
processing row-  274
processing row-  275
processing row-  567
processing row-  638


ERROR: Content Warning
If the owner of this video has granted you access, please sign in.
       This video is private.


Video missing -  638
processing row-  690
processing row-  815
processing row-  1000


ERROR: Content Warning
If the owner of this video has granted you access, please sign in.
       This video is private.


Video missing -  1000
processing row-  1020


...

Video missing -  3870
processing row-  3896
processing row-  3981
processing row-  4087


ERROR: Content Warning
If the owner of this video has granted you access, please sign in.
       This video is private.


Video missing -  4087
processing row-  4123


ERROR: Content Warning
If the owner of this video has granted you access, please sign in.
       This video is private.


Video missing -  4123
processing row-  4285
processing row-  4347
processing row-  4358
processing row-  4359
processing row-  4509
processing row-  4621
processing row-  4622
processing row-  4668
processing row-  4681
processing row-  4748
processing row-  4853
processing row-  4946
processing row-  4947
processing row-  4961
processing row-  5199


In [0]:
pickle.dump(val_data_processed,open("val_data_processed.pkl",'wb'))

Save Content to Google Drive


In [0]:
from google.colab import drive
drive.mount('/content/gdrive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls "/content/gdrive/My Drive"

'0. Imp Personal Docs'	    '7. Resumes'		  Investments
'1. Academic Certificates'  '8. Apartment docs'		  model.pkl
'2. WorkExp-Payslip'	    '9. Leo documents'		 'my photos'
'3. Income tax'		     Backups			 'Old files'
'4. Car n Bike documents'    Books			 'Personal photos'
'5. Medical reports'	    'Colab Notebooks'		  train_data.pkl
'6. Neethu docs'	    'Investment details.gsheet'   val_data.pkl


In [0]:
torch.save(train_data_processed,"/content/gdrive/My Drive/train_data.pkl")
torch.save(val_data_processed,"/content/gdrive/My Drive/val_data.pkl")

NameError: ignored

In [0]:
torch.load()

### **Deep Learning Part of the Code**


In [0]:
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras import backend as K
#K.set_image_dim_ordering('tf')

### Train of the Model Starts Here

In [0]:
train_df['text_no'] = train_df['text'].apply(data_classes.index)
train_labels = train_df['text_no'].tolist()

NameError: ignored

In [0]:


#with open("train_data_processed.pkl", "rb") as f:
train_data  = torch.load("/content/gdrive/My Drive/train_data.pkl")
train_images,train_labels =train_data

train_labels = train_df['text_no']


#with open("val_data_processed.pkl", "rb") as f:
#  val_data = pickle.load(f)
val_data = torch.load("/content/gdrive/My Drive/val_data.pkl")
val_images, val_labels = val_data


train_images = np.reshape(train_images, (train_images.shape[0], IMAGE_SIZE, IMAGE_SIZE, 1))
val_images = np.reshape(val_images, (val_images.shape[0], IMAGE_SIZE, IMAGE_SIZE, 1))
train_labels = np_utils.to_categorical(train_labels)
val_labels = np_utils.to_categorical(val_labels)


model, callbacks_list = cnn_model()
model.summary()
model.fit(train_images, train_labels, validation_data=(val_images, val_labels), epochs=15, batch_size=500, callbacks=callbacks_list)
scores = model.evaluate(val_images, val_labels, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))
torch.save(model,"/content/gdrive/My Drive/model.pkl")

KeyError: ignored

In [0]:
def process_entire_df(df):
  """
  Loops through the entire dataframe creating avg frame
  """
  print("Total No of rows to be processed",df.shape[0])
  avg_frame_list = []
  for i, row in df.iterrows():
    print("processing row- ",i)
    frame_no = row['end']
    video_url = row['url']
    #label = row['text']
    try:
      np_avg_frame = pre_process(video_url,frame_no)
    except:
      np_avg_frame = np.zeros((IMAGE_SIZE,IMAGE_SIZE))
      print("Video missing - ",i)
    avg_frame_list.append(np_avg_frame)
  return np.array(avg_frame_list)


In [0]:
def pre_process(video_url,frame_no):
    """
    :param video url frame_no
    :return:  average frame pixel values after greyscale and image resizing
    """

    vPafy = pafy.new(video_url)
    play = vPafy.getbest(preftype="any")

    #start the video
    cap = cv2.VideoCapture(play.url)
    fgbg = cv2.createBackgroundSubtractorMOG2() #Back ground Subtraction
    #print(video_url)
    processed_frame=[]
    for i in range(0,frame_no):
      ret, frame = cap.read()
      #cv2_imshow(frame)
      # Display the resulting frame
      img_array = frame.astype('uint8')
     
      #frame = fgbg.apply(frame) #remove Background
      
      
      #img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
      img_array = cv2.resize(img_array, (IMAGE_SIZE, IMAGE_SIZE))
      #cv2_imshow(img_array)
      
      # Reshape array to l * w * channels
      #img_array = img_array.reshape(IMAGE_SIZE, IMAGE_SIZE, 1)

      # Normalize the array
      #img_array = img_array / 255.0

      # Expand Dimension of the array as our model expects a 4D array
      #img_array = np.expand_dims(img_array, axis=0)
      processed_frame.append(img_array)
    process_frame_np =np.array(processed_frame)
   
    return process_frame_np 
 

In [0]:

def cnn_model():
	num_of_classes = get_num_of_classes()
 
  model = Sequential()
	model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
										input_shape=(None, IMAGE_SIZE, IMAGE_SIZE, 1),
										padding='same', return_sequences=True))
	model.add(BatchNormalization())

	model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
										padding='same', return_sequences=True))
	model.add(BatchNormalization())

	model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
										padding='same', return_sequences=True))
	model.add(BatchNormalization())

	model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
										padding='same', return_sequences=True))
	model.add(BatchNormalization())

	model.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
								activation='sigmoid',
								padding='same', data_format='channels_last'))
  model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(num_of_classes, activation='softmax'))
	#sgd = optimizers.SGD(lr=1e-2)
	model.compile(loss='binary_crossentropy', optimizer='adadelta')
  return model
 


In [0]:




	model = Sequential()
	model.add(Conv2D(16, (2,2), input_shape=(IMAGE_SIZE, IMAGE_SIZE, 1), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
	model.add(Conv2D(32, (3,3), activation='relu'))
	model.add(MaxPooling2D(pool_size=(3, 3), strides=(3, 3), padding='same'))
	model.add(Conv2D(64, (5,5), activation='relu'))
	model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(num_of_classes, activation='softmax'))
	sgd = optimizers.SGD(lr=1e-2)
	model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
	filepath="cnn_model_keras2.h5"
	checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
	callbacks_list = [checkpoint1]
	#from keras.utils import plot_model
	#plot_model(model, to_file='model.png', show_shapes=True)
	return model, callbacks_list

In [0]:
def get_num_of_classes():
	return len(train_df['text'].unique())
 

Train Code 


In [0]:
val_labels = np_utils.to_categorical([1,2])

In [0]:
val_labels

array([[0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

In [0]:
!pip install pafy
!pip install youtube-dl

     |████████████████████████████████| 1.8MB 2.8MB/s 


In [0]:
!pip install videofeatures

In [0]:
!pip install pipeline

In [0]:
image=cv2.resize(image,(224,224), interpolation = cv2.INTER_AREA)
cv2.imwrite(filename,image)

In [0]:
help(model.MaxPooling3D)

AttributeError: ignored

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, ConvLSTM2D,BatchNormalization,MaxPooling3D

In [0]:

model = Sequential()


In [0]:
from videofeatures import VGGFeatures, SIFTFeatures, SURFFeatures

In [0]:
#from videofeatures import Pipeline, ActivityNetDataset, ResNetFeatures